# Welcome to the Lab 🥼🧪
## How do create supply/demand analysis for many markets with great charts?

In this notebook, we will retrieve all markets currently tradeable on the [Parcl Exchange](https://app.parcl.co/collection/active-markets) and do a supply/demand analysis

**Note** This notebook will work with any of the 70k+ markets in the API

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/supply_demand_charts.ipynb)

In [ ]:
import os
import sys
import json
import subprocess
from datetime import datetime
from urllib.request import urlopen

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# Initialize the Parcl Labs client
client = ParclLabsClient(api_key)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

import plotly.express as px
import plotly.graph_objects as go

labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}

# Set charting constants
labs_logo_dict = dict(
    source=labs_logo_lookup['white'],
    xref="paper",
    yref="paper",
    x=0.5,  # Centering the logo below the title
    y=1.04,  # Adjust this value to position the logo just below the title
    sizex=0.15, 
    sizey=0.15,
    xanchor="center",
    yanchor="bottom"
)

def build_chart(name, data, save_graphic=True):
    HEIGHT = 900
    WIDTH = 1600
    
    fig = px.line(
        data,
        x='date',
        y='Number of New Listings',
        color='Rolling Window (Days)',
        title=f'New Listings (Rolling Day Counts): {name}',
        labels={'count': 'Rolling Day Count'}
    )

    fig.add_layout_image(
        labs_logo_dict
    )

    color_map = {
        '30 Days': '#448CF2',
        '90 Days': '#FFFFFF',
    }

    fig.update_traces(
        line=dict(width=3),
        mode='lines+markers',
        marker=dict(size=5)
    )

    for trace in fig.data:
        if trace.name in color_map:
            trace.line.color = color_map[trace.name]
            trace.marker.color = color_map[trace.name]

    # Adding gridlines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    
    fig.add_layout_image(labs_logo_dict)
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=110, b=0),
        height=HEIGHT,
        width=WIDTH,
        title={
            'text': f'New Listings (Rolling Counts): {name}',
            'y': 0.99,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#000000',  # Dark background for better contrast
        paper_bgcolor='#000000',  # Dark background for the paper
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text='',
            showgrid=False,  # Disable vertical grid lines
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        yaxis=dict(
            title_text='Number of New Listings',
            showgrid=True,
            gridwidth=0.5,  # Horizontal grid line width
            gridcolor='rgba(255, 255, 255, 0.2)',  # Horizontal grid line color with opacity
            tickfont=dict(size=14),
            tickprefix='',  # Add dollar sign to y-axis labels
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        hovermode='x unified',  # Unified hover mode for better interactivity
        hoverlabel=dict(
            bgcolor='#1F1F1F',
            font_size=14,
            font_family="Rockwell"
        )
    )
    
    # Show the plot
    if save_graphic:
        fig.write_image(os.path.join('../graphics/pricefeeds/supply_demand', f'{name}_supply_demand.png'), width=WIDTH, height=HEIGHT)
    fig.show()

def format_names(nme):
    state = nme.split(',')[-1].strip().upper().split('-')[0]
    metro = nme.split(',')[0].split('-')[0].strip()
    metro = metro.split('/')[0].strip()
    return f"{metro}, {state}"


def calculate_percent_changes(data, raw=False):
    # Ensure the date column is in datetime format
    data['date'] = pd.to_datetime(data['date'])
    
    # Sort the data by date
    data = data.sort_values(by='date').reset_index(drop=True)
    
    # Get the start and current price_feed values
    start_price = data.iloc[0]['price_feed']
    current_price = data.iloc[-1]['price_feed']
    
    # Define the date ranges
    now = data.iloc[-1]['date']
    five_years_ago = now - timedelta(days=5*365)
    one_year_ago = now - timedelta(days=365)
    six_months_ago = now - timedelta(days=6*30)
    thirty_days_ago = now - timedelta(days=30)
    
    # Helper function to get the price at a specific date
    def get_price_at_date(date):
        filtered_data = data[data['date'] <= date]
        if not filtered_data.empty:
            return filtered_data.iloc[-1]['price_feed']
        else:
            return None

    # Get the prices at the specified dates
    price_5_years_ago = get_price_at_date(five_years_ago)
    price_1_year_ago = get_price_at_date(one_year_ago)
    price_6_months_ago = get_price_at_date(six_months_ago)
    price_30_days_ago = get_price_at_date(thirty_days_ago)

    # Calculate percent changes
    def percent_change(old, new, raw=raw):
        if old is not None and new is not None:
            change = ((new - old) / old)
            emoji = '📈' if change > 0 else '📉'
            if raw:
                return change
            else:
                change = change * 100
                return f"{change:.2f}% {emoji}"
        else:
            return None
    
    changes = {
        '% Change (30 Day)': percent_change(price_30_days_ago, current_price, raw=raw),
        '% Change (6 mo)': percent_change(price_6_months_ago, current_price, raw=raw),
        '% Change (YoY)': percent_change(price_1_year_ago, current_price, raw=raw),
        '% Change (5 yr)': percent_change(price_5_years_ago, current_price, raw=raw),
        '% Change (Since `10)': percent_change(start_price, current_price, raw=raw),
        
        
    }
    
    return changes


In [ ]:
# lets get all US markets currently available to trade on the Parcl Exchange
# Now lets say you want all price feed markets that are on the parcl exchange
market_df = client.search_markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    sort_order='DESC',
    as_dataframe=True,
    # query='Tampa',
    params={'limit': 14},  # expand the default limit to 14, as of this writing, 14 markets are available
)

# lets store the parcl_ids of the markets we are interested in
parcl_ids = market_df['parcl_id'].tolist()

In [ ]:
# first lets get listings viewed as rolling counts over 30, 60, 90 day windows
start_date = '2023-01-01'
listings = client.for_sale_market_metrics_new_listings_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=start_date,
    as_dataframe=True,
    params={
        'limit': 300
    }
)
listings.head()

stock = client.market_metrics_housing_stock.retrieve_many(
    parcl_ids=parcl_ids,
    params={
        'limit': 1 # get last value, i.e. current housing stock
    },
    as_dataframe=True
)

counts = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    params={
        'limit': 1 # get last value, i.e. current housing stock
    },
    as_dataframe=True
)

In [ ]:
# 5/20/24 is last date
# 5/15/23 is YoY benchmark
listings_current = listings.loc[listings['date']=='2024-05-20'][['rolling_30_day', 'parcl_id']]
listings_yoy = listings.loc[listings['date'] == '2023-05-15'][['rolling_30_day', 'parcl_id']]
listings_yoy = listings_yoy.rename(columns={'rolling_30_day': 'YoY'})
yoy = pd.merge(listings_current, listings_yoy, on='parcl_id')
yoy['New Listings YoY % Change (30 Day Rolling)'] = ((yoy['rolling_30_day'] - yoy['YoY']) / yoy['YoY'])
yoy.head()

In [ ]:
cnts = counts[['sales', 'new_listings_for_sale', 'parcl_id', 'date']]
stk = stock[['parcl_id', 'all_properties']]
cnts = cnts.merge(stk, on='parcl_id')
cnts['delta'] = cnts['new_listings_for_sale'] - cnts['sales']
cnts = cnts.merge(yoy[['New Listings YoY % Change (30 Day Rolling)', 'parcl_id']], on='parcl_id')
cnts = cnts.merge(market_df[['parcl_id', 'name']], on='parcl_id')

In [ ]:
# melt it for charting
listings_long = pd.melt(
    listings, 
    id_vars=['date', 'parcl_id'], 
    value_vars=['rolling_30_day', 'rolling_60_day', 'rolling_90_day'],
    value_name='Number of New Listings',
    var_name='Rolling Window (Days)'
)

listings_long = listings_long.merge(market_df[['parcl_id', 'name']], on='parcl_id')
listings_long['Rolling Window (Days)'] = listings_long['Rolling Window (Days)'].replace({
    'rolling_30_day': '30 Days',
    'rolling_60_day': '60 Days',
    'rolling_90_day': '90 Days'
})

In [ ]:
pid = 5384169

# build_chart('texas', listings_long.loc[listings_long['parcl_id']==pid])

In [ ]:
for pid in listings_long.sort_values('name')['parcl_id'].unique():
    data = listings_long.loc[listings_long['parcl_id'] == pid].sort_values('date')
    sum_data = cnts.loc[cnts['parcl_id']==pid]
    name = data['name'].iloc[0].replace('Kings County', 'Brooklyn County').replace('Washington City', 'Washington, DC')
    build_chart(name, data, save_graphic=True)

    april_new_listings = sum_data['new_listings_for_sale'].values[0]
    april_sales = sum_data['sales'].values[0]
    delta = sum_data['delta'].values[0]
    pct_change_yoy = sum_data['New Listings YoY % Change (30 Day Rolling)'].values[0]
    print(name)
    print(f"April New Listings: {april_new_listings}")
    print(f"April Sales: {april_sales}")
    inc_decr = 'Increase' if delta > 0 else 'Decrease'
    print(f"Delta: {delta} Total Supply {inc_decr}")
    print(f"New Listings YoY % Change (30 Day Rolling): {pct_change_yoy:.02%}\n")
    print('Trade today on: @parcl')
    # create row

In [ ]:
cnts.head()

output = cnts[['name', 'new_listings_for_sale', 'sales', 'delta', 'New Listings YoY % Change (30 Day Rolling)']]
output = output.rename(columns={
    'new_listings_for_sale': 'April `24 New Listings for Sale',
    'sales': 'April `24 Sales',
    'delta': 'April `24 Supply Delta',
    'New Listings YoY % Change (30 Day Rolling)': 'New Listings YoY % Change (30 Day Rolling)'
})

output.head()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming the DataFrame `df` is already defined and populated with the relevant data
df = output[['name', 'April `24 New Listings for Sale', 'April `24 Sales', 'April `24 Supply Delta', 'New Listings YoY % Change (30 Day Rolling)']]
df['New Listings YoY % Change (30 Day Rolling)'] = df['New Listings YoY % Change (30 Day Rolling)']*100

# Function to format the supply delta with a plus sign if positive
def format_supply_delta(value):
    return f"+{value}" if value > 0 else str(value)

# Function to format the percentage with 2 decimal places
def format_percentage(value):
    return f"{value:.2f}%"

# Format the supply delta and percentage change columns
df['April `24 Supply Delta'] = df['April `24 Supply Delta'].apply(format_supply_delta)
df['New Listings YoY % Change (30 Day Rolling)'] = df['New Listings YoY % Change (30 Day Rolling)'].apply(format_percentage)

# Prepare data for the table
formatted_data = [df[col].tolist() for col in df.columns]

# Define headers and table layout
column_headers = ['<b>Market</b>', '<b>April `24 New Listings for Sale</b>', '<b>April `24 Sales</b>', '<b>April `24 Supply Delta</b>', '<b>New Listings YoY % Change (30 Day)</b>']

fig = go.Figure(data=[go.Table(
    header=dict(values=column_headers,
                fill_color='#000000',  # Black for header
                font=dict(color='#FFFFFF', size=12),
                align='center',
                height=30),
    cells=dict(values=formatted_data,
               fill_color='#000000',  # No color coding for cells
               font=dict(color='#FFFFFF', size=12),
               align='center',
               height=30)
)])

# Add the logo image
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}
labs_logo_dict = dict(
    source=labs_logo_lookup['white'],
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.01,
    sizex=0.2,
    sizey=0.2,
    xanchor="center",
    yanchor="bottom"
)
fig.add_layout_image(labs_logo_dict)

# Set the dimensions of the figure
w = 1400
h = 600

# Update layout and display the figure
fig.update_layout(
    title={
        'text': 'Parcl Exchange Markets Supply & Demand Overview',
        'y': 0.94,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font_color='#FFFFFF',
    width=w,  # Increase the width for wider cells
    height=h,
    paper_bgcolor='#080D16',
    margin=dict(l=10, r=10, t=100, b=10)
)

fig.show()

fig.write_image(os.path.join('../graphics/pricefeeds/supply_demand', f'comp_table_supply_demand.png'), width=w, height=h)
